**This notebook is derived from the tutorial on [Data Cleaning](https://www.kaggle.com/learn/data-cleaning) by [Rachael Tatman](https://www.kaggle.com/rtatman) at [kaggle](https://www.kaggle.com/)**

# Inconsistent Data Entry

## Importing Libraries

In [2]:
import pandas as pd
import numpy as np

import fuzzywuzzy
from fuzzywuzzy import process
import chardet

import warnings
warnings.filterwarnings('ignore')

# Loading Dataset

In [8]:
data_professors = pd.read_csv("pakistan_intellectual_capital.csv")

## Perform some initial text per-processing
Take a look at first few rows of the data.

In [9]:
data_professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


Consider the "Country" column, we need to clean up this column by making sure that there are no inconsistent entries in it. Manual check row-by-row is very tideous and time consuming process, but there are always ways to break complex problem to a simple solution.

We'll get all unique values in "Country" column then sort them alphabetically, then take a closser look :

In [12]:
countries = data_professors["Country"].unique()
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

Just by looking at this, we can spot the inconsistencies like 'Germany' & 'germany' or 'New Zealand' & 'New Zealand'.

Let's make everything lower case and remove any white spaces at the beginning & end of cells. nconsistencies in capitalizations and trailing white spaces are very common in text data and almost 80% of inconsistencies can be fixed by doing this.

In [13]:
# Convert lower-case
data_professors['Country'] = data_professors["Country"].str.lower()

# Remove white spcaes
data_professors['Country'] = data_professors['Country'].str.strip()

## Use fuzzy matching to correct inconsistent data entry
Next we'll fix more difficult inconsistencies. Take a look at "Country" column again:

In [14]:
countries = data_professors["Country"].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

There is another inconsistency : 'southkorea' and 'south korea', both should be same.

Although this dataset is small enough to  correct errors by hand, but that approach doesn't scale (when we're dealing with a dataset with thousands of observations). Fuzzywuzzy package can identify which strings are closest to each other, that's what we're going to use.
> **Fuzzy Matching:** The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.

Fuzzywuzzy returns a ratio of two given strings, closer the ratio to 100 smaller the edit difference between the two strings.

In [15]:
# Top 10 closest matches to "south korea"
matches = fuzzywuzzy.process.extract('south korea', countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('ireland', 33),
 ('portugal', 32),
 ('singapore', 30),
 ('netherland', 29),
 ('macau', 25),
 ('usofa', 25)]

We can see that two of the items in the cities are very close to "south korea": "south korea" and "southkorea". Let's replace all rows in our "Country" column that have a ratio of > 47 with "south korea".

Create a reusable function to replace rows in the provided column of the provided dataframe,
that match the provided string above the provided ratio with the provided string :

In [27]:
def replace_matches(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get top 10 matches
    matches = fuzzywuzzy.process.extract('usa', countries, limit=10, 
                                         scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    # only get matches with ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    
    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
    
    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, column] = string_to_match
    
    print("Done!")

Let's check if our function works properly or not:

In [18]:
replace_matches(df=data_professors, column="Country", string_to_match="south korea")

Done!


Now once again let's take a look at unique values of "Country" column to check if our data inconsistency is fixed or not:


In [19]:
countries = data_professors["Country"].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

Notice that 'southkorea' is now replaced with 'south korea'. 

# Exercise:

# Examine another column
Now let's take a look at all unique values in "Graduated from" column:

In [21]:
graduated_from = data_professors["Graduated from"].unique()
graduated_from.sort()
graduated_from

array([' Columbia University', ' Delft University of Technology',
       ' Iowa State University', ' University of Central Florida',
       ' University of Innsbruck',
       ' University of Texas at Arlington (UTA)', ' University of Turin',
       'Abasyn University', 'Abdul Wali Khan University, Mardan',
       'Abdus Salam School of Mathematical Sciences,GC University',
       'Agricultural University Peshawar', 'Allama Iqbal Open University',
       'Asian Institute of Technology', 'Aston University, Birmingham',
       'Australian National University, Caneberra', 'BUKC',
       'Bahauddin Zakariya University', 'Bahria University',
       'Bahria University,Islamabad',
       'Balochistan University of Information Technology, Engineering and Management Sciences',
       'Barani Institute of Information Technology',
       'Beaconhouse National University', 'Beihang University',
       'Beijing Institute of Technology',
       'Beijing Institute of Technology Beijing',
       'Beiji

# Perform text pre-processing 
Convert every entry in the "Graduated from" column in the professors DataFrame to remove white spaces at the beginning and end of cells:

In [22]:
data_professors['Graduated from'] = data_professors['Graduated from'].str.strip()

Let's take a look at "Graduated from" again :

In [23]:
graduated_from = data_professors["Graduated from"].unique()
graduated_from.sort()
graduated_from

array(['Abasyn University', 'Abdul Wali Khan University, Mardan',
       'Abdus Salam School of Mathematical Sciences,GC University',
       'Agricultural University Peshawar', 'Allama Iqbal Open University',
       'Asian Institute of Technology', 'Aston University, Birmingham',
       'Australian National University, Caneberra', 'BUKC',
       'Bahauddin Zakariya University', 'Bahria University',
       'Bahria University,Islamabad',
       'Balochistan University of Information Technology, Engineering and Management Sciences',
       'Barani Institute of Information Technology',
       'Beaconhouse National University', 'Beihang University',
       'Beijing Institute of Technology',
       'Beijing Institute of Technology Beijing',
       'Beijing University of Posts & Telecommunications',
       'Biztek Institute Of Business & Technology,Karachi',
       'Blekinge Institute of Technology', 'Boston University',
       'Brock University Canada', 'Brunel University',
       'CECOS Uni

# Cleaning Country further
There are still some inconsistencies in "Country" column, like: 'usa' and 'usofa' should be same.

In [25]:
countries = data_professors["Country"].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

Lets find the closest mathces to the 'usa' using Fuzzywuzzy:

In [26]:
matches = fuzzywuzzy.process.extract('usa', countries, limit=10, 
                                         scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('usa', 100),
 ('usofa', 75),
 ('austria', 60),
 ('australia', 50),
 ('spain', 50),
 ('urbana', 44),
 ('uk', 40),
 ('malaysia', 36),
 ('pakistan', 36),
 ('portugal', 36)]

Use the replace_matches() function to fix this inconsistency:

First edit the matches as 'usa' in the function, then call the replace_matches()

In [28]:
replace_matches(df=data_professors, column="Country", string_to_match="usa")

Done!


Let's take a look at "Country": 

In [29]:
countries = data_professors["Country"].unique()
countries.sort()
countries

array(['canada', 'china', 'finland', 'france', 'germany', 'greece',
       'hongkong', 'ireland', 'italy', 'japan', 'macau', 'malaysia',
       'mauritius', 'netherland', 'new zealand', 'norway', 'pakistan',
       'portugal', 'russian federation', 'saudi arabia', 'scotland',
       'singapore', 'south korea', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa'], dtype=object)

All fixed, 'usofa' is replaced by usa.